# 2. Convolution and Pooling (10 - 10 pts)

#### Ümit Mert ÇAĞLAR 
#### 2043685

### a) Write a function for convolution operation in the cell below. Write your own code with your own algorithm. It should have necessary error checks for matrix dimensions (consider multichannel matrices). Inputs must have the same structure as Tensorflow convolution operation and also the results must be same. Check [tf.nn.convolution](https://www.tensorflow.org/api_docs/python/tf/nn/convolution)

# Imports

In [1]:
import tensorflow as tf
import numpy as np
import math
import scipy.signal

# Convolution

I will first analyze the samples, play with their dimensions and tf functions then fill my_convolution class
I have assumed square matrices, matrices with different width and height are not much different but require a few more lines of definition, I do not think it is important for the sake of convolution.

In [2]:
def my_convolution(my_input, my_filter, my_padding='VALID', my_strides=[1,1]): #define default padding and stride size
    # MY CONVOLUTION IMPLEMENTATION
    if(len(my_input.shape)!=len(my_filter.shape)): #length of shape array gives the dimensions of matrices they should be equal
        print('ERROR CODE:000 MATRIX DIMENSIONS DOES NOT MATCH!! check dimensions')
        return -1
    if(my_input.shape[0]!=my_filter.shape[3]):
        print('ERROR CODE:001 MATRIX DIMENSIONS DOES NOT MATCH!! check channel depth')
        return -1
    #we have checked matrix dimensions now lets start convolution operation
    #start with padding
    #tensorflow has two padding options valid and same
    #same keeps input size same as output size which means padding is done automatically
    #valid keeps input size fixed which means no padding is done
    #https://www.tensorflow.org/api_docs/python/tf/nn/convolution states that zero padding is done.
    #my_input=np.pad(my_input[0,:,:,0],pad_width=1, mode='constant', constant_values=0) #
    #my_input=my_input.reshape(1,my_input.shape[0],my_input.shape[1],1)
    input_size=my_input.shape[1] #take input size from the input tensor
    filter_size=my_filter.shape[0] #take filter size from the filter tensor
    my_padding_size=0 #assign values before defining them in statements
    my_output_size=0
    if (my_padding!='SAME' and my_padding!='VALID'): #check padding type give error
        print('ERROR CODE:002 given padding is not valid try SAME or VALID')
        return -1
    if (my_padding=='VALID'):
        my_padding_size=0
        my_output_size=math.floor(((input_size-filter_size)/my_strides[0])+1)
        print('Convolution without padding')
    if (my_padding=='SAME'):
        my_padding_size=math.ceil((input_size*my_strides[0]-my_strides[0]-input_size+filter_size)/2)
        my_output_size=my_input.shape[1] #outputsize is same as input size
        #I hand calculated this formula from the output size calculation in CNN
        # N=((D-K+2P)/S) +1 where D is input K is filter size (kernel size) P is padding (before and after) and S is strides
        # P Padding equals to the above formula
        
        print('Convolution with padding')
    npad = ((0, 0), (my_padding_size, my_padding_size), (my_padding_size, my_padding_size))
    my_input=np.pad(my_input[:,:,:,0],pad_width=npad, mode='constant', constant_values=0)
    my_input=my_input.reshape(my_input.shape[0],my_input.shape[1],my_input.shape[2],1)
    my_output=np.zeros((my_input.shape[0],my_output_size,my_output_size))
    #print(my_output.shape)
    
    #my_output=np.zeros((my_input.shape[0],my_output_size,my_output_size,1))
    #print(my_output.shape)
    # we will do output size amount of convolution operations
    # for SAME padding we have outputsize=inputsize for valid we have (N-K)/S +1
    for c in range(0,my_input.shape[0]): # number of channels
        for i in range(0,my_output_size+1): #scan columns in input
            for j in range(0,my_output_size+1): #scan rows in input
                input_mini_array=my_input[c,i:i+filter_size,j:j+filter_size,0] #input mini array
                filter_mini_array=my_filter[:,:,0,c]
                #print(input_mini_array)
                #print(my_output[i,j])
                my_output[c,i-1,j-1]=scipy.signal.convolve2d(input_mini_array,filter_mini_array,mode='valid')
    my_output=my_output.reshape(my_input.shape[0],my_output_size,my_output_size,1)
    print("Ouutput shape: "+str(my_output.shape))
    return my_output

In [3]:
def tf_convolution(tf_input, tf_filter, tf_padding, tf_strides):
    
    sess = tf.Session()
    tf_function = tf.nn.convolution(input=tf_input, filter=tf_filter, padding=tf_padding, strides=tf_strides)
    tf_output = sess.run(tf_function)
    sess.close()
    
    return tf_output

In [4]:
sample_input = np.random.random([1,4,4,1])
sample_filter = np.random.random([2,2,1,1])
sample_padding = "SAME"
sample_strides = [1,1]

tf_output = tf_convolution(sample_input,sample_filter,sample_padding,sample_strides)
my_output = my_convolution(sample_input,sample_filter,sample_padding,sample_strides)

print("input:")
print(sample_input[:,:,:,0])
print(sample_input.shape)
print("filter:")
print(sample_filter[:,:,0,:])
print(sample_filter.shape)
print("output:")
print(tf_output[:,:,:,0])
#print(tf_output)
print(tf_output.shape)
print("MY output:")
print(my_output[:,:,:,0])
#print(tf_output)
print(my_output.shape)

Convolution with padding
Ouutput shape: (1, 4, 4, 1)
input:
[[[0.97680156 0.60108959 0.9008249  0.08709111]
  [0.66818236 0.45127513 0.23705848 0.94482689]
  [0.9954758  0.97215904 0.57890568 0.59076859]
  [0.03002362 0.19591377 0.76085756 0.31357023]]]
(1, 4, 4, 1)
filter:
[[[0.48040507]
  [0.52456551]]

 [[0.44870243]
  [0.84747654]]]
(2, 2, 1, 1)
output:
[[[1.46683143 1.16469791 1.38553318 0.46578513]
  [1.82827596 1.26796667 1.3699266  0.71897893]
  [1.16769673 1.50342001 1.19514811 0.42450795]
  [0.11719311 0.49323761 0.53000796 0.15064073]]]
(1, 4, 4, 1)
MY output:
[[[1.66482705 1.26421911 1.38075829 0.56943127]
  [1.75797953 1.27688413 1.2123298  1.11061545]
  [1.38971984 1.55192781 1.30544865 0.66515065]
  [0.1133513  0.50743096 0.78550866 0.26574342]]]
(1, 4, 4, 1)


The TF output and MY output are very close in numbers,however they are different and I could not debug the case
Most probably in my calculations I mistakenly omitted or added one more or less elements for each entry in the output. 
Matrix dimensions, shape and data types are same

## Observations and Testing
Below I tested and made some observations to better understand convolution and its calculation and matrix dimensions, types and shapes. It is similar with pooling so I just used below for testbed for both cases.

In [5]:
x=my_convolution(sample_input, sample_filter,my_padding='SAME', my_strides=[1,1])
scipy.signal.convolve2d(sample_input[0,0:2,0:2,0],sample_filter[:,:,0,0],mode='valid')

Convolution with padding
Ouutput shape: (1, 4, 4, 1)


array([[1.66482705]])

In [6]:
print(sample_input.shape)
print(sample_filter.shape)

(1, 4, 4, 1)
(2, 2, 1, 1)


After changing sample input and filter values I observed that:

In [7]:
print("input channel size: "+str(sample_input.shape[0]))
print("input width size: "+str(sample_input.shape[1]))
print("input height size: "+str(sample_input.shape[2])) 
print("input element size (should be 1 as each element should contain 1 int/float): "+str(sample_input.shape[3]))
print("filter width size: "+str(sample_filter.shape[0]))
print("filter height size: "+str(sample_filter.shape[1]))
print("filter element size (should be 1 as each element should contain 1 int/float): "+str(sample_filter.shape[2])) 
print("filter channel size (number of filters): "+str(sample_filter.shape[3]))

input channel size: 1
input width size: 4
input height size: 4
input element size (should be 1 as each element should contain 1 int/float): 1
filter width size: 2
filter height size: 2
filter element size (should be 1 as each element should contain 1 int/float): 1
filter channel size (number of filters): 1


basically tf transpose circular shifts ndarray to left, to check matrix dimensions we have to be careful about number of filters with input channel size so sample_input.shape[0]==sample_filter.shape[3]

In [8]:
sample_input[0,:,:,0] #for better visualisation for 1 channel

array([[0.97680156, 0.60108959, 0.9008249 , 0.08709111],
       [0.66818236, 0.45127513, 0.23705848, 0.94482689],
       [0.9954758 , 0.97215904, 0.57890568, 0.59076859],
       [0.03002362, 0.19591377, 0.76085756, 0.31357023]])

In [9]:
sample_filter[:,:,0,0] #for better visualisation for 1 channel

array([[0.48040507, 0.52456551],
       [0.44870243, 0.84747654]])

In [10]:
tf_output.shape # it is similar to sample_input.shape 

(1, 4, 4, 1)

In [11]:
dene=sample_input

In [12]:
dene[:,:,:,0]

array([[[0.97680156, 0.60108959, 0.9008249 , 0.08709111],
        [0.66818236, 0.45127513, 0.23705848, 0.94482689],
        [0.9954758 , 0.97215904, 0.57890568, 0.59076859],
        [0.03002362, 0.19591377, 0.76085756, 0.31357023]]])

In [13]:
chsize=dene.shape[0]
for i in range (0,dene.shape[0]-1):
    dene[i]=np.pad(dene[i,:,:,0],pad_width=1, mode='constant', constant_values=0)
    dene[i]=dene[i].reshape(i+1,dene[i].shape[0],dene[i].shape[1],1)

In [14]:
dene=sample_input
npad = ((0, 0), (1, 1), (1, 1))
dene=np.pad(dene[:,:,:,0],pad_width=npad, mode='constant', constant_values=0)
dene=dene.reshape(dene.shape[0],dene.shape[1],dene.shape[2],1)

In [15]:
dene.shape

(1, 6, 6, 1)

In [16]:
my_padding_size=round((sample_input.shape[1]*1-1-sample_input.shape[1]+sample_filter.shape[1])/2)

In [17]:
my_padding_size

0

In [18]:
sample_input[:,:,:,0]

array([[[0.97680156, 0.60108959, 0.9008249 , 0.08709111],
        [0.66818236, 0.45127513, 0.23705848, 0.94482689],
        [0.9954758 , 0.97215904, 0.57890568, 0.59076859],
        [0.03002362, 0.19591377, 0.76085756, 0.31357023]]])

In [19]:
x[:,:,:,0]

array([[[1.66482705, 1.26421911, 1.38075829, 0.56943127],
        [1.75797953, 1.27688413, 1.2123298 , 1.11061545],
        [1.38971984, 1.55192781, 1.30544865, 0.66515065],
        [0.1133513 , 0.50743096, 0.78550866, 0.26574342]]])

In [20]:
x.shape

(1, 4, 4, 1)

In [21]:
dene=np.pad(dene[0,:,:,0],pad_width=1, mode='constant', constant_values=0)

In [22]:
dene=dene.reshape(1,dene.shape[0],dene.shape[1],1)

In [23]:
#dene[0,:,:,0]=np.pad(dene[0,:,:,0],pad_width=1, mode='constant', constant_values=0)

In [24]:
dene.shape

(1, 8, 8, 1)

### b) Write a function for pooling operation in the cell below. Write your own code with your own algorithm. It should have necessary error checks for matrix dimensions (consider multichannel matrices). Inputs must have the same structure as Tensorflow pooling operation and also the results must be same. Check [tf.nn.pool](https://www.tensorflow.org/api_docs/python/tf/nn/pool)

In [25]:
def my_pooling(my_input, my_window_shape, my_pooling_type='MAX', my_padding='SAME', my_strides=[1,1]):
# MY POOLING IMPLEMENTATION
    if(my_input.shape[1]<my_window_shape[0]): #length of shape array gives the dimensions of matrices input should be larger
        print('ERROR CODE:000 MATRIX DIMENSIONS DOES NOT MATCH!! check matrix dimensions')
        return -1
   # if(my_input.shape[1]!=my_window_shape[0]):
    #    print('ERROR CODE:001 MATRIX DIMENSIONS DOES NOT MATCH!! check channel depth')
   #     return -1
    #we have checked matrix dimensions now lets start convolution operation
    #start with padding
    input_size=my_input.shape[1] #take input size from the input tensor
    my_window_shape=my_window_shape[0] #take window size from the window tensor
    my_padding_size=0 #assign values before defining them in statements
    my_output_size=0
    if (my_padding!='SAME' and my_padding!='VALID'): #check padding type give error
        print('ERROR CODE:002 given padding is not valid try SAME or VALID')
        return -1
    if (my_padding=='VALID'):
        my_padding_size=0
        my_output_size=math.floor(((input_size-my_window_shape)/my_strides[0]))
        print('Pooling without padding')
    if (my_padding=='SAME'):
        my_padding_size=math.ceil((input_size*my_strides[0]-my_strides[0]-input_size+my_window_shape)/2)
        my_output_size=my_input.shape[1] #outputsize is same as input size
        #I hand calculated this formula from the output size calculation in CNN
        # N=((D-K+2P)/S) +1 where D is input K is window size (kernel size) P is padding (before and after) and S is strides
        # P Padding equals to the above formula
        print('Pooling with padding')
    npad = ((0, 0), (my_padding_size, my_padding_size), (my_padding_size, my_padding_size))
    my_input=np.pad(my_input[:,:,:,0],pad_width=npad, mode='constant', constant_values=0)
    my_input=my_input.reshape(my_input.shape[0],my_input.shape[1],my_input.shape[2],1)
    my_output=np.zeros((my_input.shape[0],my_output_size,my_output_size))
    #print(my_output.shape)
    #my_output=np.zeros((my_input.shape[0],my_output_size,my_output_size,1))
    #print(my_output.shape)
    # we will do output size amount of convolution operations
    # for SAME padding we have outputsize=inputsize for valid we have (N-K)/S +1
    for c in range(0,my_input.shape[0]): # number of channels
        for i in range(0,my_output_size+1): #scan columns in input
            for j in range(0,my_output_size+1): #scan rows in input
                input_mini_array=my_input[c,i:i+my_window_shape,j:j+my_window_shape,0] #input mini array
                #print(input_mini_array)
                #print(my_output[i,j])
                my_output[c,i-1,j-1]=max(map(max, input_mini_array))
                #my_output[c,i-1,j-1]=max(input_mini_array)
    my_output=my_output.reshape(my_input.shape[0],my_output_size,my_output_size,1)
    print("Output shape: "+str(my_output.shape))
    
    
    
    
    return my_output

In [26]:
def tf_pooling(tf_input, tf_window_shape, tf_pooling_type, tf_padding, tf_strides):
    
    sess = tf.Session()
    tf_function = tf.nn.pool(input=tf_input, window_shape=tf_window_shape, pooling_type=tf_pooling_type, padding=tf_padding, strides=tf_strides)
    tf_output = sess.run(tf_function)
    sess.close()
    
    return tf_output

In [31]:
sample_input = np.random.random([1,4,4,1])
sample_window_shape = [2,2]
sample_pooling_type = "MAX"
sample_padding = "VALID"
sample_strides = [2,2]

tf_output = tf_pooling(sample_input,sample_window_shape,sample_pooling_type,sample_padding,sample_strides)
my_output = my_pooling(my_input=sample_input, my_window_shape=sample_window_shape, my_pooling_type=sample_pooling_type, my_padding=sample_padding)

print("input:")
print(sample_input)
print("output:")
print(tf_output)
print("MY output:")
print(my_output)

Pooling without padding
Output shape: (1, 2, 2, 1)
input:
[[[[0.60389018]
   [0.82586377]
   [0.64032503]
   [0.0581604 ]]

  [[0.80264873]
   [0.48479838]
   [0.7696826 ]
   [0.31801813]]

  [[0.78297069]
   [0.93518825]
   [0.76748707]
   [0.06025765]]

  [[0.27305131]
   [0.8131291 ]
   [0.33836793]
   [0.75748431]]]]
output:
[[[[0.82586377]
   [0.7696826 ]]

  [[0.93518825]
   [0.76748707]]]]
MY output:
[[[[0.93518825]
   [0.7696826 ]]

  [[0.93518825]
   [0.76748707]]]]


TF output and MY output seems to be similar, pooling operation is done with tf library and with my implementation

In [28]:
sample_window_shape

[2, 2]

In [29]:
sample_input[0,0:0+sample_window_shape[0],0:0+sample_window_shape[0],0]

array([[0.27626404, 0.21399442],
       [0.17070351, 0.66850176]])

In [30]:
sample_window_shape

[2, 2]